In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
import pickle


In [2]:
# Load Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


4422102/4422102 [==============================] - 0s 0us/step


In [3]:
# Reshape input images to include a channel dimension
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)


In [4]:
# Convert grayscale images to RGB format
train_images = np.repeat(train_images, 3, axis=-1)
test_images = np.repeat(test_images, 3, axis=-1)


In [5]:
# Resize images to meet the input shape requirement of (32, 32, 3)
train_images_resized = [array_to_img(img).resize((32, 32)) for img in train_images]
test_images_resized = [array_to_img(img).resize((32, 32)) for img in test_images]


In [6]:
# Convert resized images back to numpy arrays and normalize pixel values to [0, 1]
train_images_resized = np.array([img_to_array(img) for img in train_images_resized]) / 255.0
test_images_resized = np.array([img_to_array(img) for img in test_images_resized]) / 255.0


In [7]:
# Load pre-trained MobileNetV2 model without top layer
base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')


9406464/9406464 [==============================] - 0s 0us/step


In [8]:
# Add new top layers for classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(10, activation='softmax')(x)



In [9]:
# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)


In [10]:
# Unfreeze the top layers of MobileNetV2
for layer in base_model.layers:
    layer.trainable = True  # Set layer.trainable to True to unfreeze the layer


In [11]:
# Compile the model with a lower learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [15]:
# Train the model with a lower learning rate and more epochs
history = model.fit(train_images_resized, train_labels, epochs=2, batch_size=32, validation_data=(test_images_resized, test_labels))

Epoch 1/2
1875/1875 [==============================] - 379s 202ms/step - loss: 0.5133 - accuracy: 0.8166 - val_loss: 0.6533 - val_accuracy: 0.7494
Epoch 2/2
1875/1875 [==============================] - 375s 200ms/step - loss: 0.4447 - accuracy: 0.8453 - val_loss: 0.5685 - val_accuracy: 0.8107


In [16]:
# Save the trained model to a pkl file
with open('MNISTmodel.pkl', 'wb') as f:
    pickle.dump(model, f)